In [2]:
import pandas as pd
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
%reload_ext jupyternotify

<IPython.core.display.Javascript object>

In [3]:
def addBravaisLattice(dataframe, spaceGroupTag):
    spaceGroupDf = pd.read_excel("Book1.xlsx")
    spaceGroupDf.columns = ['Crystal System','Space Group','Full Name']
    spaceGroupDf['Bravais Lattice'] = spaceGroupDf['Full Name'].str[0] + '-' + spaceGroupDf['Crystal System'].astype(str) 
    merged = dataframe.merge(spaceGroupDf, left_on=spaceGroupTag, right_on='Space Group', how='left')
    dataframe = merged.drop(['Bravais','Space Group','Full Name'],axis=1)
    
    merged.loc[merged['Bravais Lattice'] == 'P-tetragonal', 'Bravais Lattice'] = 'P-Tetragonal'
    merged.loc[merged['Bravais Lattice'] == 'I-tetragonal', 'Bravais Lattice'] = 'I-Tetragonal'
    merged.loc[merged['Bravais Lattice'] == 'P-Trigonal', 'Bravais Lattice'] = 'R-Trigonal'
    merged.loc[merged['Bravais Lattice'] == 'A-Orthorhombic', 'Bravais Lattice'] = 'C-Orthorhombic'
    merged.loc[merged['Bravais Lattice'] == 'P-hexagonal', 'Bravais Lattice'] = 'P-Hexagonal'
    return merged

In [4]:
pathToICSD = "ICSD_index (1).csv"
ICSD = pd.read_csv(pathToICSD)
ICSD = addBravaisLattice(ICSD, 'sym_group')

In [5]:
CSD = pd.read_csv('CSD_With_Bravais_Lattices.csv')

In [6]:
COD = pd.read_csv('COD.csv')

/tmp/ipykernel_3258554/3367538459.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  COD = pd.read_csv('COD.csv')


In [12]:
COD.iloc[:50].sort_values(by=['Chemical Sum', 'Point Group'])


KeyError: 'Chemical Sum'

In [ ]:
CSD.iloc[:25].sort_values(by=['Chemical Sum', 'Space Group IT Number'])


,name,cif_names,publication_year,ICSD_creation_date,Bravais,sym_group,a,b,c,alpha,beta,gamma,V,Crystal System,Space Group,Full Name,Bravais Lattice
0,Cr2O11Te4,1,1976,1986/07/25,monoclinic,14,7.0160,7.5450,9.7280,90.0,99.69,90.0,507.61,Monoclinic,14,P21/c,P-Monoclinic
1,Co0.93Mn1.07O4Si1,2,1988,2002/04/01,orthorhombic,62,10.5100,6.1290,4.8230,90.0,90.00,90.0,310.68,Orthorhombic,62,Pnma,P-Orthorhombic
2,F3La1,3,1976,1985/08/14,trigonal,165,7.1850,7.1850,7.3510,90.0,90.00,120.0,328.65,Trigonal,165,P3c1,R-Trigonal
3,Ce1F3,4,1976,1980/01/01,trigonal,165,7.1310,7.1310,7.2860,90.0,90.00,120.0,320.86,Trigonal,165,P3c1,R-Trigonal
4,H4Na1O5P1,5,1976,1980/01/01,orthorhombic,33,7.6160,7.8990,7.3820,90.0,90.00,90.0,444.09,Orthorhombic,33,Pna21,P-Orthorhombic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181357,C4H16Cl2Mn1N8S4,966808,2014,2015/02/01,tetragonal,86,13.7526,13.7526,9.0785,90.0,90.00,90.0,1717.05,Tetragonal,86,P42/n,P-Tetragonal
181358,Fe1Se1,977900,2014,2015/02/01,orthorhombic,67,5.3104,5.3317,5.4879,90.0,90.00,90.0,155.38,Orthorhombic,67,Cmme,C-Orthorhombic
181359,Fe1Se1,977901,2014,2015/02/01,tetragonal,129,3.8280,3.8280,5.5821,90.0,90.00,90.0,81.80,Tetragonal,129,P4/nmm,P-Tetragonal
181360,Fe0.96Se1,977902,2014,2015/02/01,hexagonal,194,3.7565,3.7565,5.9621,90.0,90.00,120.0,72.86,Hexagonal,194,P63/mmc,P-Hexagonal


In [20]:
ICSD.iloc[135:160].sort_values(by=['name', 'sym_group'])

,name,cif_names,publication_year,ICSD_creation_date,Bravais,sym_group,a,b,c,alpha,beta,gamma,V,Crystal System,Space Group,Full Name,Bravais Lattice
140,Al1O9P3,170,1976,1980/01/01,monoclinic,9,10.4230,18.6870,9.2220,90.00,81.630,90.00,1777.08,Monoclinic,9,Cc,C-Monoclinic
145,Ba1Ni1O3,175,1976,1980/01/01,hexagonal,194,5.6290,5.6290,4.8110,90.00,90.000,120.00,132.02,Hexagonal,194,P63/mmc,P-Hexagonal
156,Ba2Mn1O6W1,189,1976,1980/01/01,cubic,225,8.1930,8.1930,8.1930,90.00,90.000,90.00,549.96,Cubic,225,Fm3m,F-Cubic
137,Bi5Cu1Pb1S9,167,1976,1980/01/01,orthorhombic,62,33.5310,11.4860,4.0030,90.00,90.000,90.00,1541.70,Orthorhombic,62,Pnma,P-Orthorhombic
150,C1.484B13.777,180,1975,1980/01/01,hexagonal,166,5.6720,5.6720,12.1428,90.00,90.000,120.00,338.32,Trigonal,166,R3m,R-Trigonal
147,C10H20K6O34U2,177,1976,1980/01/01,triclinic,2,10.1030,10.9440,10.0210,121.40,104.700,63.80,848.16,Triclinic,2,P1,P-Triclinic
151,C6D6Cl8Fe3N6,182,1975,1980/01/01,trigonal,147,10.2900,10.2900,6.2830,90.00,90.000,120.00,576.14,Trigonal,147,P3,R-Trigonal
141,Cl1Fe2O4P1,171,1976,1986/08/04,monoclinic,15,13.6770,9.2170,9.3260,90.00,132.530,90.00,866.36,Monoclinic,15,C2/c,C-Monoclinic
148,Cu1K2N6O12Sr1,178,1976,1980/01/01,orthorhombic,69,10.4301,10.4621,10.9368,90.00,90.000,90.00,1193.43,Orthorhombic,69,Fmmm,F-Orthorhombic
139,Cu3Na2O12Si4,169,1976,1980/01/01,orthorhombic,62,7.5190,10.2030,13.5670,90.00,90.000,90.00,1040.81,Orthorhombic,62,Pnma,P-Orthorhombic


### Combining ICSD, COD, and CSD

1. COD: Fix chemical_sum name and strip all the whitespaces in COD to match ICSD
2. ICSD: Add path column and move cif_names there
3. ICSD and COD: Add 'Database' column thats either ICSD or COD
4. Make sure columns are the same

List of Final Columns
'name', 'Local Path', 'Database', 'Publication Year', 'a', 'b','c','alpha','beta','gamma','Volume', 'Bravais Lattice', 'Space Group','Numeric Bravais Lattice'

'name' = chemical name w no spaces 

'local path' = within the folder of the dataspace, this is a path to the file(inside the parent folder)

'publication year' = year it was published

'Bravais Lattice' = String bravais Lattice

'Space group' = numerical space group

'Numerical Space Group' = 0-13 for spacing group, see mapping below


In [ ]:
#Task 1
COD['name'] = COD['Chemical Sum'].str.replace(' ', '')
CSD['name'] = CSD['Chemical Sum'].str.replace(' ', '')

In [ ]:
#Task 2
#ISCD['Local Zip Path']
ICSD['Local Path'] = "diffraction.icsd_" +  ICSD['cif_names'].apply(lambda x: '{:06d}'.format(int(x)))
CSD['Local Path'] = CSD['File Path']
COD['Local Path'] = COD['Local Zip Path']

In [ ]:
#Task 3
ICSD['Database'] = 'ICSD'
COD['Database'] = 'COD'
CSD['Database'] = 'CSD'

In [ ]:
COD['Space Group'] = COD['Space Group IT Number']
ICSD['Space Group'] = ICSD['sym_group']
CSD['Space Group'] = CSD['Space Group IT Number']

In [ ]:
COD['Volume'] = COD['volume']
ICSD['Volume'] = ICSD['V']
CSD['Volume'] = CSD['Cell Volume']

CSD['a'] = CSD['Cell Length A']
CSD['b'] = CSD['Cell Length B']
CSD['c'] = CSD['Cell Length C']
CSD['alpha'] = CSD['Cell Angle Alpha']
CSD['beta'] = CSD['Cell Angle Beta']
CSD['gamma'] = CSD['Cell Angle Gamma']

ICSD['Publication Year'] = ICSD['publication_year']
COD['Publication Year'] = COD['pubYear']

In [ ]:
COD = COD[['name', 'Local Path', 'Database', 'Publication Year', 'a', 'b','c','alpha','beta','gamma','Volume', 'Bravais Lattice', 'Space Group']]
ICSD = ICSD[['name', 'Local Path', 'Database', 'Publication Year', 'a', 'b','c','alpha','beta','gamma','Volume', 'Bravais Lattice', 'Space Group']]
CSD = CSD[['name', 'Local Path', 'Database', 'Publication Year', 'a', 'b','c','alpha','beta','gamma','Volume', 'Bravais Lattice', 'Space Group']]

In [ ]:
#CrystalData = pd.concat([COD, ICSD,CSD], axis=0)
CrystalData = pd.concat([CSD,ICSD],axis=0)
cols_to_check = [col for col in CrystalData.columns if col != 'Publication Year']

In [ ]:
mapping = {0: 'F-Cubic',
  1: 'I-Cubic',
  2: 'P-Cubic',
  3: 'P-Hexagonal',
  4: 'C-Monoclinic',
  5: 'P-Monoclinic',
  6: 'C-Orthorhombic',
  7: 'F-Orthorhombic',
  8: 'I-Orthorhombic',
  9: 'P-Orthorhombic',
  10: 'R-Trigonal',
  11: 'I-Tetragonal',
  12: 'P-Tetragonal',
  13: 'P-Triclinic'}
my_dict_rev = {v: k for k, v in mapping.items()}
ICSD['Numeric Bravais Lattice'] = ICSD['Bravais Lattice'].map(my_dict_rev)
ICSD.to_csv('ICSD_With_Bravais.csv')

In [ ]:
CrystalData.to_csv('Crystal Database.csv')

In [ ]:
CrystalData = pd.read_csv('Crystal Database.csv')
ignore_column = 'Publication Year'

# Drop rows with NaN values, ignoring the specified column
df_cleaned = CrystalData.dropna(subset=[col for col in CrystalData.columns if col != ignore_column])

In [ ]:
CrystalData = df_cleaned
CrystalData